# Causal Inference with Pyoso

Welcome to this tutorial on using causal inference with pyoso! This tutorial will guide you through the process of using synthetic control methods to analyze the causal impact of events or interventions on open source projects using data from OSO (Open Source Observer).

## Overview

Causal inference goes beyond correlation to understand cause-and-effect relationships. While randomized experiments are the gold standard for establishing causation, they aren't always possible in the context of open source software development. This is where observational causal inference methods become valuable.

In this tutorial, we'll explore:

1. Introduction to causal inference and why it matters for open source metrics
2. Setting up your environment with pyoso
3. Understanding synthetic control methods
4. Implementing a synthetic control analysis to measure the impact of grant funding on open source projects
5. Interpreting and visualizing your results
6. Best practices and limitations

## 1. Introduction to Causal Inference

Causal inference is the process of determining whether a specific intervention or treatment had an effect on an outcome of interest. The fundamental challenge of causal inference is that we can never observe the counterfactual - what would have happened if the treatment had not been applied.

In open source software metrics, we might want to answer questions like:

- Did receiving a grant increase the number of active developers contributing to a project?
- Does a specific contribution behavior (like posting) lead to increased long-term engagement?
- What impact did a specific event (like a hackathon) have on project activity?

These questions are inherently causal. Simple correlational analysis might lead to misleading conclusions because of confounding factors - variables that affect both the treatment and the outcome.

## 2. Setting Up Your Environment

First, let's install the necessary packages and set up our environment:

In [ ]:
# Install required packages
!pip install pyoso pysyncon numpy pandas matplotlib seaborn

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyoso import Client
import pysyncon as psc

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
# Connect to OSO with your API key
# If you're running this in Google Colab, you can use userdata.get('OSO_API_KEY')
# If running locally, you can use environment variables or paste your key directly (not recommended for shared notebooks)

try:
    from google.colab import userdata
    OSO_API_KEY = userdata.get('OSO_API_KEY')
except Exception:
    OSO_API_KEY = os.environ.get('OSO_API_KEY')  # Set your API key as an environment variable

# Initialize the client
client = Client(api_key=OSO_API_KEY)

## 3. Understanding Synthetic Control Methods

### What are Synthetic Control Methods?

The synthetic control method is a statistical technique used to evaluate the effect of an intervention in comparative case studies. It creates a weighted combination of control units (the "synthetic control") that best resembles the characteristics of the treated unit before the intervention.

The key idea is to use data from before the intervention to create a synthetic version of the treated unit, and then compare the post-intervention outcomes between the actual treated unit and its synthetic counterpart. The difference between these two represents the causal effect of the intervention.

### When to Use Synthetic Control Methods?

Synthetic control methods are particularly useful when:

1. You have a small number of treated units (often just one)
2. You have several potential control units
3. You have data for time periods before and after the intervention
4. Randomized experiments are not feasible

This makes them perfect for analyzing the impact of events or interventions on open source projects.

## 4. Implementing a Synthetic Control Analysis

Let's use a concrete example: analyzing the impact of receiving a grant on the number of active developers contributing to a project.

### Step 1: Define the research question

We want to know: **Did receiving a grant increase the number of active developers for a project?**

### Step 2: Identify the treated and potential control units

- Treated unit: A project that received a grant at a specific point in time
- Potential control units: Similar projects that did not receive grants

### Step 3: Collect data on the outcome variable and predictors before and after the intervention

Let's query OSO data to get timeseries metrics for active developers:

In [ ]:
# Define our helper function for formatting arrays in SQL queries
def stringify(arr):
    return "'" + "','".join(arr) + "'"

In [ ]:
# For this example, let's assume a project called 'project-xyz' received a grant in January 2024
# We'll compare it with other similar projects that didn't receive grants

# First, let's identify our treated project and potential control projects
treated_project = 'project-xyz'  # Replace with an actual project name from the OSO database
potential_control_projects = ['project-a', 'project-b', 'project-c', 'project-d', 'project-e']

# Let's get active developer data for these projects
query = f"""
SELECT
  p.project_name,
  tm.sample_date,
  tm.amount as active_devs_90d
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m ON m.metric_id = tm.metric_id
JOIN projects_v1 AS p ON p.project_id = tm.project_id
WHERE
  p.project_name IN ({stringify([treated_project] + potential_control_projects)})
  AND m.metric_name = 'active_developers_over_90_day'
  AND tm.sample_date BETWEEN '2023-01-01' AND '2024-06-30'
ORDER BY p.project_name, tm.sample_date
"""

df_devs = client.to_pandas(query)
df_devs.head()

In [ ]:
# Let's also get other metrics that might be good predictors
predictors = ['commits_over_90_day', 'issues_opened_over_90_day', 'new_contributors_over_90_day']

query = f"""
SELECT
  p.project_name,
  m.metric_name,
  tm.sample_date,
  tm.amount
FROM timeseries_metrics_by_project_v0 AS tm
JOIN metrics_v0 AS m ON m.metric_id = tm.metric_id
JOIN projects_v1 AS p ON p.project_id = tm.project_id
WHERE
  p.project_name IN ({stringify([treated_project] + potential_control_projects)})
  AND m.metric_name IN ({stringify(predictors)})
  AND tm.sample_date BETWEEN '2023-01-01' AND '2024-06-30'
ORDER BY p.project_name, m.metric_name, tm.sample_date
"""

df_predictors = client.to_pandas(query)
df_predictors.head()

In [ ]:
# Reshape the predictor data from long to wide format
df_predictors_wide = df_predictors.pivot_table(
    index=['project_name', 'sample_date'],
    columns='metric_name',
    values='amount'
).reset_index()

# Merge with our active developer data
df_combined = pd.merge(
    df_devs,
    df_predictors_wide,
    on=['project_name', 'sample_date'],
    how='inner'
)

df_combined.head()

### Step 4: Define the pre-intervention and post-intervention periods

For our example, let's assume the project received a grant on January 1, 2024:

In [ ]:
# Define intervention date
intervention_date = '2024-01-01'

# Plot the data for the treated project with a vertical line at the intervention date
plt.figure(figsize=(14, 8))
for project in [treated_project] + potential_control_projects:
    project_data = df_combined[df_combined['project_name'] == project]
    plt.plot(project_data['sample_date'], project_data['active_devs_90d'], label=project)

plt.axvline(x=pd.to_datetime(intervention_date), color='r', linestyle='--', alpha=0.7, label='Intervention (Grant)')
plt.title('Active Developers (90-day) Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Active Developers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Step 5: Prepare the data for pysyncon

The pysyncon package requires data in a specific format. Let's prepare our data:

In [ ]:
# Prepare data for pysyncon
# First, reshape the data to have dates as rows and projects as columns
pivot_df = df_combined.pivot(index='sample_date', columns='project_name', values='active_devs_90d')
pivot_df.index = pd.to_datetime(pivot_df.index)
pivot_df = pivot_df.sort_index()

# Prepare predictor variables similarly
predictor_dfs = {}
for predictor in predictors:
    pred_pivot = df_combined.pivot(index='sample_date', columns='project_name', values=predictor)
    pred_pivot.index = pd.to_datetime(pred_pivot.index)
    pred_pivot = pred_pivot.sort_index()
    predictor_dfs[predictor] = pred_pivot

# Convert to numpy arrays for pysyncon
treated_outcome = pivot_df[treated_project].values
control_outcomes = pivot_df.drop(columns=[treated_project]).values
control_names = pivot_df.drop(columns=[treated_project]).columns.tolist()

# Find the index of the intervention date
intervention_idx = pivot_df.index.get_loc(pd.to_datetime(intervention_date))

# Extract pre-intervention and post-intervention periods
pre_period = (0, intervention_idx - 1)  # Index 0 to the day before intervention
post_period = (intervention_idx, len(pivot_df) - 1)  # Intervention day to the end

# Prepare predictor matrices
predictor_matrices = []
for predictor in predictors:
    pred_treated = predictor_dfs[predictor][treated_project].values
    pred_controls = predictor_dfs[predictor].drop(columns=[treated_project]).values
    predictor_matrices.append((pred_treated, pred_controls))

### Step 6: Run the synthetic control analysis

In [ ]:
# Create and fit the synthetic control model
synth = psc.psc.Synth()
synth.fit(
    X1=treated_outcome[pre_period[0]:pre_period[1]+1],  # Pre-intervention treated outcome
    X0=control_outcomes[pre_period[0]:pre_period[1]+1],  # Pre-intervention control outcomes
    Z1=[p[0][pre_period[0]:pre_period[1]+1] for p in predictor_matrices],  # Pre-intervention treated predictors
    Z0=[p[1][pre_period[0]:pre_period[1]+1] for p in predictor_matrices],  # Pre-intervention control predictors
)

# Get the weights assigned to each control unit
synth_weights = synth.weights
synth_control_weights = pd.DataFrame({'Project': control_names, 'Weight': synth_weights})
synth_control_weights = synth_control_weights.sort_values('Weight', ascending=False)

# Display the weights
print("Synthetic Control Weights:")
print(synth_control_weights)

# Plot the weights
plt.figure(figsize=(10, 6))
sns.barplot(x='Weight', y='Project', data=synth_control_weights, palette='viridis')
plt.title('Synthetic Control Weights by Project')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the synthetic control series
synthetic_outcome = np.dot(control_outcomes, synth_weights)

# Create a DataFrame for plotting
results_df = pd.DataFrame({
    'Date': pivot_df.index,
    'Actual': treated_outcome,
    'Synthetic': synthetic_outcome,
    'Gap': treated_outcome - synthetic_outcome
})

# Mark pre and post intervention periods
results_df['Period'] = 'Pre-intervention'
results_df.loc[results_df['Date'] >= pd.to_datetime(intervention_date), 'Period'] = 'Post-intervention'

# Plot the results
plt.figure(figsize=(14, 8))
plt.subplot(2, 1, 1)
plt.plot(results_df['Date'], results_df['Actual'], 'b-', label='Actual')
plt.plot(results_df['Date'], results_df['Synthetic'], 'r--', label='Synthetic Control')
plt.axvline(x=pd.to_datetime(intervention_date), color='green', linestyle='--', alpha=0.7, label='Intervention (Grant)')
plt.title(f'Synthetic Control Analysis: Impact of Grant on {treated_project}')
plt.ylabel('Active Developers (90-day)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot the gap (treatment effect)
plt.subplot(2, 1, 2)
plt.bar(results_df['Date'], results_df['Gap'], alpha=0.7, label='Treatment Effect (Gap)')
plt.axvline(x=pd.to_datetime(intervention_date), color='green', linestyle='--', alpha=0.7, label='Intervention (Grant)')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.title('Estimated Treatment Effect (Gap between Actual and Synthetic Control)')
plt.ylabel('Effect Size')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Step 7: Evaluate the fit and calculate the treatment effect

In [ ]:
# Calculate the mean pre-intervention fit
pre_MSE = np.mean((results_df[results_df['Period'] == 'Pre-intervention']['Actual'] - 
                   results_df[results_df['Period'] == 'Pre-intervention']['Synthetic']) ** 2)
pre_RMSE = np.sqrt(pre_MSE)
print(f"Pre-intervention RMSE: {pre_RMSE:.4f}")

# Calculate the average treatment effect in the post-intervention period
average_treatment_effect = np.mean(results_df[results_df['Period'] == 'Post-intervention']['Gap'])
print(f"Average Treatment Effect: {average_treatment_effect:.4f} additional active developers")

# Calculate the percentage change
average_synthetic = np.mean(results_df[results_df['Period'] == 'Post-intervention']['Synthetic'])
percentage_change = (average_treatment_effect / average_synthetic) * 100
print(f"Percentage Change: {percentage_change:.2f}%")

### Step 8: Conduct placebo tests to assess the significance of our findings

To evaluate whether our results could have occurred by chance, we can conduct placebo tests by applying the same analysis to each control unit as if it had received the treatment:

In [ ]:
# Function to run placebo test for a single control unit
def run_placebo_test(control_idx, treated_outcome, control_outcomes, predictor_matrices, pre_period, post_period):
    # Extract the control unit to be treated as "treated"
    placebo_treated = control_outcomes[:, control_idx]
    
    # Create a new control set excluding the placebo treated unit
    placebo_controls = np.delete(control_outcomes, control_idx, axis=1)
    
    # Add the original treated unit to the control set
    placebo_controls = np.column_stack((treated_outcome, placebo_controls))
    
    # Adjust predictor matrices
    placebo_predictor_matrices = []
    for p_treated, p_controls in predictor_matrices:
        placebo_p_treated = p_controls[:, control_idx]
        placebo_p_controls = np.delete(p_controls, control_idx, axis=1)
        placebo_p_controls = np.column_stack((p_treated, placebo_p_controls))
        placebo_predictor_matrices.append((placebo_p_treated, placebo_p_controls))
    
    # Create and fit the synthetic control model
    placebo_synth = psc.psc.Synth()
    try:
        placebo_synth.fit(
            X1=placebo_treated[pre_period[0]:pre_period[1]+1],
            X0=placebo_controls[pre_period[0]:pre_period[1]+1],
            Z1=[p[0][pre_period[0]:pre_period[1]+1] for p in placebo_predictor_matrices],
            Z0=[p[1][pre_period[0]:pre_period[1]+1] for p in placebo_predictor_matrices],
        )
        
        # Calculate the synthetic control series
        placebo_synthetic = np.dot(placebo_controls, placebo_synth.weights)
        
        # Calculate the gap (effect)
        placebo_gap = placebo_treated - placebo_synthetic
        
        return placebo_gap
    except Exception:
        # If the placebo test fails, return None
        return None

In [ ]:
# Run placebo tests for all control units
placebo_gaps = []
placebo_labels = []

# Real treated unit first
placebo_gaps.append(results_df['Gap'].values)
placebo_labels.append(treated_project)

# Then all placebo units
for i in range(len(control_names)):
    gap = run_placebo_test(i, treated_outcome, control_outcomes, predictor_matrices, pre_period, post_period)
    if gap is not None:
        placebo_gaps.append(gap)
        placebo_labels.append(control_names[i])

# Plot all gaps
plt.figure(figsize=(14, 8))
for i, gap in enumerate(placebo_gaps):
    if placebo_labels[i] == treated_project:
        plt.plot(results_df['Date'], gap, 'b-', linewidth=2, label=placebo_labels[i] + " (Treated)")
    else:
        plt.plot(results_df['Date'], gap, 'gray', alpha=0.5, linewidth=1, label=placebo_labels[i])

plt.axvline(x=pd.to_datetime(intervention_date), color='green', linestyle='--', alpha=0.7, label='Intervention (Grant)')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.title('Placebo Tests: Gap between Actual and Synthetic Control for All Units')
plt.ylabel('Gap (Effect Size)')
plt.xlabel('Date')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Interpreting and Visualizing the Results

Let's interpret the results of our synthetic control analysis:

1. **Pre-intervention fit**: The RMSE in the pre-intervention period tells us how well our synthetic control matches the treated unit before the intervention. A low RMSE indicates a good fit.

2. **Average Treatment Effect**: This represents the average difference between the actual outcome and the synthetic control in the post-intervention period. If positive, it suggests the intervention had a positive effect.

3. **Percentage Change**: This gives us the relative size of the effect compared to what would have happened without the intervention.

4. **Placebo Tests**: By comparing the gap for our treated unit with placebo gaps for control units, we can assess whether the observed effect is significantly different from what we might observe by chance. If the treated unit's gap is consistently larger than the placebo gaps, it supports the conclusion that the intervention had a causal effect.

### Causal Interpretation

Based on our analysis, we can conclude whether receiving a grant causally impacted the number of active developers for our project. However, some important caveats to consider:

1. **Assumptions**: The synthetic control method assumes that the relationship between the treated unit and controls in the pre-intervention period would have continued in the absence of treatment.

2. **Confounding Events**: If other events occurred around the same time as our intervention and affected only the treated unit, they could confound our results.

3. **Extrapolation**: We should be cautious about extrapolating the results too far into the future.

4. **Number of Control Units**: The reliability of the synthetic control increases with the number of suitable control units available.

## 6. Best Practices and Limitations

### Best Practices

1. **Choose predictors wisely**: Select predictors that are likely to influence the outcome variable but aren't affected by the intervention.

2. **Check the pre-intervention fit**: A good synthetic control should closely match the treated unit in the pre-intervention period.

3. **Conduct placebo tests**: These help assess whether the observed effect could have occurred by chance.

4. **Consider multiple periods**: If possible, examine the effect over different time horizons to see if it's stable.

5. **Be transparent about limitations**: Acknowledge the assumptions and potential confounders in your analysis.

### Limitations

1. **Cannot handle multiple treated units easily**: The standard synthetic control method is designed for a single treated unit.

2. **Requires a reasonable number of pre-intervention periods**: Without sufficient pre-intervention data, it's hard to create a good synthetic control.

3. **Assumes no spillover effects**: The method assumes the intervention doesn't affect the control units.

4. **Selection of control units is crucial**: The quality of the synthetic control depends on having suitable control units.

## 7. Summary and Next Steps

In this tutorial, we've learned how to:

1. Connect to OSO data using pyoso
2. Understand the principles of causal inference and synthetic control methods
3. Prepare data for a synthetic control analysis
4. Implement a synthetic control analysis using pysyncon
5. Interpret and visualize the results
6. Evaluate the robustness of our findings through placebo tests

### Next Steps

To further explore causal inference with OSO data, you might want to:

1. **Try different outcome variables**: Beyond active developers, consider other metrics like commits, issues, or user engagement.

2. **Explore different interventions**: Such as marketing campaigns, hackathons, or new feature releases.

3. **Implement other causal inference methods**: Such as difference-in-differences, regression discontinuity, or instrumental variables.

4. **Combine with other data sources**: Integrate OSO data with other sources for a more comprehensive analysis.

